In [12]:
%matplotlib inline
from qiskit import Aer, IBMQ, execute
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister
from qiskit.tools.visualization import plot_histogram
from qiskit.tools.monitor import job_monitor
import numpy as np
from qiskit.providers.aer.noise import NoiseModel
from qiskit.test.mock import FakeYorktown
import matplotlib.pyplot as plt
import pandas as pd

In [13]:
device_backend = FakeYorktown()
coupling_map = device_backend.configuration().coupling_map
noise_model = NoiseModel.from_backend(device_backend)
basis_gates = noise_model.basis_gates

Mathematica-optimised angles:
$\theta_0 = -1.5708,\\
\theta_1 = 6.75662\text{e}-9,\\
\theta_2 = -0.979924,\\
\theta_3 = -3.14159,\\
\theta_4 = -1.51454\text{e}-8,\\
\theta_5 = 3.14159,\\
\theta_6 = 3.14159,\\
\theta_7 = 3.14159$

In [53]:
hyperparams = [-np.pi/2, 0, -0.979924, -np.pi, 0, np.pi, np.pi, np.pi]
shots = 100000
U = 4
mu = U/2
eps2 = 0
V = 0.745356

In [15]:
def fresh_circuit(theta):
    # Construct quantum circuit
    circ = QuantumCircuit(4, 4)
    circ.ry(theta[0],0)
    circ.ry(theta[1],1)
    circ.ry(theta[2],2)
    circ.ry(theta[3],3)
    circ.cx(0,1)
    circ.cx(1,2)
    circ.cx(2,3)
    circ.ry(theta[4],0)
    circ.ry(theta[5],1)
    circ.ry(theta[6],2)
    circ.ry(theta[7],3)
    return circ

In [16]:
def get_sigma(basis,index, result_noise):
    if basis == 'x':
        result = result_noise[0]
    elif basis == 'y':
        result = result_noise[1]
    elif basis == 'z':
        result = result_noise[2]

    vec = np.array([(-1)**(sum([int(n[i]) for i in index])) for n in result.keys()])
    result = np.array(list(result.values()))
    return (np.dot(result,vec)/shots)

In [51]:
circ_x = QuantumCircuit(2,2)
circ_y = QuantumCircuit(2,2)
circ_z = QuantumCircuit(2,2)
circ_x.x(0)
circ_y.x(0)
circ_y.h([0,1])
circ_z.x(0)
circ_z.sdg([0,1])
circ_z.h([0,1])
circ_x.measure([0,1],[0,1])
circ_y.measure([0,1],[0,1])
circ_z.measure([0,1],[0,1])
simulator = Aer.get_backend('statevector_simulator')
result_noise = execute([circ_x, circ_y, circ_z], backend=simulator).result().get_statevector(circ_z)

In [49]:
result_noise

array([0.+0.j, 1.+0.j, 0.+0.j, 0.+0.j])

In [17]:
def calc_H(th):
    # Select the QasmSimulator from the Aer provider

    circ_z = fresh_circuit(th)
    circ_z.measure(list(range(4)),list(range(4)))
    circ_x = fresh_circuit(th)
    circ_x.h([0,1,2,3])
    circ_x.measure(list(range(4)),list(range(4)))
    circ_y = fresh_circuit(th)
    circ_y.sdg([0,1,2,3])
    circ_y.h([0,1,2,3])
    circ_y.measure(list(range(4)),list(range(4)))
    # Execute noisy simulation and get counts
    simulator = Aer.get_backend('qasm_simulator')
    #result_noise = execute([circ_x, circ_y, circ_z], simulator).result()
    #simulator = Aer.get_backend('statevector_simulator')
    
    result_noise = execute([circ_x, circ_y,circ_z], simulator,
                           noise_model=noise_model,
                           coupling_map=coupling_map,
                           basis_gates=basis_gates,
                           shots=shots).result().get_counts()
    #reverse the order of qubits to something that makes sense
    rn = list(map(lambda x: dict(zip(list(map(lambda y: y[::-1],x.keys())), x.values() )), result_noise))

    
    H1 = U*get_sigma('z',[0,2],rn)/4 + (mu/2 - U/4)*(get_sigma('z',[0],rn) + get_sigma('z',[2],rn))
    #H2 = eps2*(sigma['z'][1]+sigma['z'][3])/2
    H2 = 0
    H3 = V*(get_sigma('x',[0,1],rn) + get_sigma('y',[0,1],rn))/2
    H4 = V*(get_sigma('x',[2,3],rn) + get_sigma('y',[2,3],rn))/2
    H = H1-H2+H3+H4
    return H

In [54]:
calc_H(hyperparams)
#BEST: -1.72640848544

-1.72892286632

In [30]:
deg = np.pi/180
lb=-1.5*deg
ub=1.5*deg
deltas = np.mgrid[lb:ub:5j, lb:ub:5j, lb:ub:5j, lb:ub:5j, lb:ub:5j, lb:ub:5j, lb:ub:5j, lb:ub:5j].reshape(8,-1).T
params = np.array(list(map( lambda x: x+hyperparams,deltas)))
results = np.zeros((len(deltas),1), dtype=np.float64)
for i in range(len(deltas)):
    results[i] = calc_H(params[i])
pd.DataFrame(np.append(deltas, results, axis=1),
             columns=['dth'+str(i) for i in range(1,9)]+['H']).to_csv('statevector_results.csv',index=False)


In [68]:
results = np.zeros((len(deltas),1), dtype=np.float64)
results[0] = 1.25
results

array([[1.25],
       [0.  ],
       [0.  ],
       ...,
       [0.  ],
       [0.  ],
       [0.  ]])

In [ ]:
"""
loop time = 1.079375 sec

"""

#only with decoherent errors: -1.5969232194799998
data = 
mean = np.mean(data, axis=0)
cov = np.cov(data, rowvar=0)